In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.integrate import quad
import random
import datetime as dt


# Funkcje do uzgodnienia typów

In [2]:
def t_exp(options, t_spot):
    new_options = []
    for option in options:
        option["expiry"] = (pd.to_datetime(option["expiry"]) - t_spot).days/365
        if option["expiry"] > 0:
            new_options.append(option)
    return new_options 

In [3]:
def t_exp_df(time, t_spot):
    return (pd.to_datetime(time) - t_spot).days/365

In [4]:
def convert_dataframe_to_dicts(portfolio_data, market_data, t_spot):
    options_data = pd.merge(portfolio_data, market_data, on = "fx_rate")
    options_data["expiry"] = options_data["expiry"].apply(lambda x: t_exp_df(x, t_spot))
    options_data = options_data[options_data["expiry"] > 0]
    options_data["df_f"] = np.exp(-options_data["expiry"]*options_data["r_ccy1"])
    options_data["df_d"] = np.exp(-options_data["expiry"]*options_data["r_ccy2"])
    options_data["call_put"] = options_data["call_put"].apply(lambda x: 1 if x == "call" else -1)
    options_data["position"] = options_data["position"].apply(lambda x: 1 if x == "buy" else -1)
    options_data["option_spec"] = [[options_data["instrument"][i], options_data["call_put"][i]] for i in range(len(options_data))   
    ]
    options_data = options_data.rename(columns = {"nominal_ccy2": "nominal"})
    options = options_data.to_dict("records")

    return options

# Strategie

In [5]:
def strategy(options, S, nominal_id = False):
    profit = 0
    for element in options:
        option_type, call_put = element["option_spec"]
        if nominal_id:
            position, nominal, strike = element["position"], 1, element["strike"]   
        else:
            position, nominal, strike = element["position"], element["nominal"], element["strike"]
            
        if option_type == "Vanilla":
            profit += position * nominal * max(0, call_put * (S - strike))
        elif option_type == "Con":
            profit += strike * position * nominal * (call_put * (S - strike) > 0)
        elif option_type == "Aon":
            profit += S * position * nominal * (call_put * (S - strike) > 0)
    return profit

In [6]:
def unique_strikes(options):
    strikes = [e["strike"] for e in options]
    return np.unique(strikes)

In [7]:
def plot_strategy(S_range, options, nominal_id = True, normed = True):
    
    values = np.array([strategy(options, S, nominal_id) for S in S_range])

    strikes = unique_strikes(options)
    if normed:
        values = values / max(abs(values))

    plt.figure(figsize=(8, 5))
    for strike in strikes:
        plt.axvline(x=strike, color='g', linestyle='--', alpha=0.3)
    plt.plot(S_range, values, linestyle='-', linewidth=2, color='r', label="Strategy Value")
    plt.xlabel("S", fontsize=12, fontweight='bold')
    plt.ylabel("Wypłata ze strategii", fontsize=12, fontweight='bold')
    plt.title("Wypłata ze strategii", fontsize=14, fontweight='bold')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xlim([min(S_range), max(S_range)])
    plt.ylim([-2,2])

# Funkcja rozkładu lognormalnego

In [8]:
def s_pdf(x, S, df_d, df_f, sigma, T):
    rd, rf = np.log(df_d)/T, np.log(df_f)/T
    exponent = -0.5 * ((np.log(x/S) - (rd - rf - (sigma**2)/2) * T) / (sigma * np.sqrt(T)))**2
    return (1 / (x * sigma * np.sqrt(2 * np.pi * T))) * np.exp(exponent)

In [9]:
def integrand(x, options, S, df_d, df_f, sigma, T):
    return df_d * strategy(options, x) * s_pdf(x, S, df_d, df_f, sigma, T)

In [10]:
def integrate(options, S, df_d, df_f, sigma, T):
    return quad(integrand, 0 , 1000 * sigma, args = (options, S, df_d, df_f, sigma, T ) )

# Funkcje do wyliczania wartości Blacka - Scholesa

In [11]:
def calculate_d_1(option, S, sigma, df_d, df_f):
    return (np.log(S/option["strike"])+ np.log(df_f/df_d) + (sigma**2 / 2) * option["expiry"]) / (sigma * np.sqrt(option["expiry"]))

In [12]:
def black_scholes_price(options, S, sigma, df_d, df_f):
    value = 0
    for element in options:
        opt_type, sign = element["option_spec"]
        pos, nominal, strike, expiry_date = element["position"], element["nominal"], element["strike"], element["expiry"]

        d_1 = calculate_d_1(element, S, sigma, df_d, df_f)
        d_2 = d_1 - sigma * np.sqrt(expiry_date)

        n_1 = stats.norm.cdf(sign * d_1)
        n_2 = stats.norm.cdf(sign * d_2)

        if opt_type == "Vanilla": 
            value += pos * nominal * sign * (df_f * S * n_1 - df_d * strike * n_2)
        elif opt_type == "Con":   
            value += pos * nominal * sign * (df_d * n_2)
        elif opt_type == "Aon":
            value += pos * nominal * sign * (df_f * n_1)
    return value

# Funkcje do wyliczania odpowiednich delt

In [13]:
def delta(option, S, df_f, df_d, sigma):
    if option["option_spec"][0] == "FX Fwd":
        return df_f * option["nominal"] * option["position"] / option["strike"]
    else:
        d_1 = calculate_d_1(option, S, sigma, df_d, df_f)
        
        return option["option_spec"][1] * df_f * option["nominal"] * option["position"] * stats.norm.cdf(option["option_spec"][1] * d_1) / option["strike"]


In [14]:
def cross_currency_delta(option, S, df_f, df_d, sigma):
    nominal_ccy1 = (1/option["strike"]) * option["nominal"]
    nominal_ccy2 = option["nominal"]

    if option["option_spec"][0] not in ["Vanilla", "FX Fwd"]:
        raise ValueError("Only Vanilla options/ FX Fwd should be plugged into this function")
    
    if option["option_spec"][0] == "FX Fwd":
        delta_ccy_1 = df_f * nominal_ccy1 * option["position"]
        delta_ccy_2 = - df_d * nominal_ccy2 * option["position"]

    else:
        d_1 = calculate_d_1(option, S, sigma, df_d, df_f)
        delta_ccy_1 = option["option_spec"][1] * df_f * nominal_ccy1 * option["position"] * stats.norm.cdf(option["option_spec"][1] * d_1)
        delta_ccy_2 = black_scholes_price([option], S, sigma, df_d, df_f) - S * delta_ccy_1

    return delta_ccy_1, delta_ccy_2

# Funkcje, które będą przekładały DataFramy na słowniki

In [15]:
def portfolio_delta_for_each(portfolio_data, market_data, t_spot):
    cross_currency = ["EUR/USD"]
    options = convert_dataframe_to_dicts(portfolio_data, market_data, t_spot)
    for option in options:
        if option["fx_rate"] in cross_currency:
            option["delta_ccy1"], option["delta_ccy2"] = cross_currency_delta(option, option["fx_spot"], option["df_f"], option["df_d"], option["sigma"])
        else:
            option["delta_ccy1"], option["delta_ccy2"] = delta(option, option["fx_spot"], option["df_f"], option["df_d"], option["sigma"]), 0
        option["delta_ccy1_curr"], option["delta_ccy2_curr"] = option["fx_rate"][:3], option["fx_rate"][4:]

    options_data = pd.DataFrame(options)       
    options_data.drop(columns= ["option_spec"], inplace= True) 
    return options_data

In [16]:
def portfolio_delta_all(portfolio_data, market_data, t_spot):
    data = portfolio_delta_for_each(portfolio_data, market_data, t_spot)

    grouped = (
        pd.concat([
            data.groupby("delta_ccy1_curr", as_index=False)[["delta_ccy1"]].sum().rename(
                columns={"delta_ccy1_curr": "currency", "delta_ccy1": "delta"}),
            data.groupby("delta_ccy2_curr", as_index=False)[["delta_ccy2"]].sum().rename(
                columns={"delta_ccy2_curr": "currency", "delta_ccy2": "delta"})
        ])
        .groupby("currency", as_index=False)
        .sum()
        .fillna(0)
    )
    
    return grouped

# Funkcje do policzenia zmiany wartości porfela

In [17]:
def portfolio_bs_price_for_each(portfolio_data, market_data, t_spot):
    options = convert_dataframe_to_dicts(portfolio_data, market_data, t_spot)

    for option in options:
        option["bs_price"] = black_scholes_price([option], option["strike"], option["sigma"], option["df_d"], option["df_f"])
        option["bs_currency"] = option["fx_rate"][4:]

    options_data = pd.DataFrame(options)       
    options_data.drop(columns= ["option_spec"], inplace= True) 

    return options_data

In [18]:
def portfolio_bs_price_all(portfolio_data, market_data, t_spot):
    data = portfolio_bs_price_for_each(portfolio_data, market_data, t_spot)
    grouped = data.groupby("bs_currency", as_index = False)[["bs_price"]].sum()

    return grouped


# Delta hedging

In [19]:
def harmongram(option_s, S, sigma, r_d, r_f, t_spot, step):
    option = option_s.copy()
    deltas, days, S_t, bs_price, p_l  = [], [], [], [], []
    t_spot = pd.to_datetime("2025/01/15")
    expiry_date = pd.to_datetime(option["expiry"])
    option["expiry"] = pd.to_datetime(option["expiry"])
    S_spot = S
    expiry_date_frac = (option["expiry"] - t_spot).days
    if expiry_date_frac < 0:
        raise ValueError("Option expired or step value is too big")

    for i in range(0, expiry_date_frac, step):
        
        option["expiry"] = (expiry_date_frac - i)/365
        df_d = np.exp(- option["expiry"] * r_d )
        df_f = np.exp(- option["expiry"] * r_f )

        days.append(t_spot + dt.timedelta(i))
        bs_price.append(black_scholes_price([option], S, sigma, df_d, df_f))
        deltas.append(delta(option, S, df_f, df_d, sigma))
        S_t.append(S)

        if len(p_l) == 0:
            p_l.append(bs_price[-1] - deltas[-1] * S_spot)
        else:
            p_l.append(p_l[-1] * np.exp(i * r_d ) - (deltas[-1] - deltas[-2] * np.exp(i * r_f) * S_t[-1] ))

        S = S_spot * np.exp((r_d - 0.5 * sigma**2) * ((i+1)/365) + sigma * np.random.normal(0, 1) * np.sqrt((i + 1)/365))

    S_T = S_spot * np.exp((r_d - 0.5 * sigma**2) * (expiry_date_frac/365) + sigma * np.random.normal(0, 1) * np.sqrt(expiry_date_frac/365))
    days.append(expiry_date)
    bs_price.append(black_scholes_price([option], S, sigma, df_d, df_f))
    deltas.append(delta(option, S, df_f, df_d, sigma))
    S_t.append(S_T)
    p_l.append(p_l[-1] * np.exp(r_d * (option["expiry"])) - option["nominal"] * max( option["option_spec"][1] * (S_T - option["strike"]) , 0) + deltas[-1] * np.exp(r_f * option["expiry"]) )
    return pd.DataFrame({
        "day": days,
        "S_t": np.array(S_t),
        "bs_price": np.array(bs_price),
        "delta": np.array(deltas),
        "p_l": np.array(p_l)
    })
    

In [21]:
t_spot = pd.to_datetime("2025/01/15")

data_por = pd.read_excel("fx_portfolio.xlsx", sheet_name = "Portfolio")
data_market = pd.read_excel("fx_portfolio.xlsx", sheet_name = "Market data")

data = convert_dataframe_to_dicts(data_por, data_market, t_spot)
option = data[0]
option["expiry"] = pd.to_datetime("2025/09/15")
S, sigma, r_d, r_f =  option["fx_spot"], option["sigma"] , option["r_ccy2"], option["r_ccy1"]

In [22]:
harmongram(option, S, sigma, r_d, r_f, t_spot, 10)
# plot_harmonogram(option, S, [0.04,0.05, 0.3], r_d, r_f, t_spot, 10)


,day,S_t,bs_price,delta,p_l
0,2025-01-15,4.100000,1.996406e+03,5.316924e+03,-1.980298e+04
1,2025-01-25,4.111759,2.024757e+03,5.465329e+03,-8.604433e+03
2,2025-02-04,4.097856,1.482071e+03,4.265006e+03,1.315413e+04
3,2025-02-14,4.065996,8.180799e+02,2.602627e+03,9.900327e+04
4,2025-02-24,4.127875,1.627503e+03,4.785011e+03,7.624248e+05
5,2025-03-06,4.041501,3.776396e+02,1.372597e+03,9.373533e+06
6,2025-03-16,4.274816,7.214819e+03,1.682303e+04,1.882911e+08
7,2025-03-26,4.347404,1.348059e+04,2.753230e+04,6.235915e+09
8,2025-04-05,4.110024,5.840149e+02,2.150559e+03,3.404707e+11
9,2025-04-15,4.071114,2.238951e+02,9.441497e+02,3.064819e+13


In [24]:
portfolio_delta_for_each(data_por, data_market, t_spot)

,id,fx_rate,instrument,call_put,position,nominal_ccy1,nominal,expiry,strike,fx_spot,sigma,r_ccy1,r_ccy2,df_f,df_d,delta_ccy1,delta_ccy2,delta_ccy1_curr,delta_ccy2_curr
0,1,EUR/PLN,Vanilla,1,1,100000,450000,0.665753,4.500,4.100,0.06,0.03,0.05,0.980226,0.967260,5316.923857,0.000000,EUR,PLN
1,2,EUR/PLN,Vanilla,1,1,50000,212500,0.441096,4.250,4.100,0.06,0.03,0.05,0.986854,0.978187,12557.722889,0.000000,EUR,PLN
2,3,EUR/PLN,Vanilla,1,-1,50000,222500,0.441096,4.450,4.100,0.06,0.03,0.05,0.986854,0.978187,-1717.606100,0.000000,EUR,PLN
3,4,EUR/PLN,Vanilla,-1,-1,150000,622500,0.265753,4.150,4.100,0.06,0.03,0.05,0.992059,0.986800,86465.594314,0.000000,EUR,PLN
4,5,EUR/PLN,Vanilla,-1,1,150000,630000,0.265753,4.200,4.100,0.06,0.03,0.05,0.992059,0.986800,-107588.831283,0.000000,EUR,PLN
5,14,EUR/PLN,FX Fwd,-1,1,100000,412500,0.569863,4.125,4.100,0.06,0.03,0.05,0.983049,0.971909,98304.941510,0.000000,EUR,PLN
6,15,EUR/PLN,FX Fwd,-1,-1,75000,313125,0.342466,4.175,4.100,0.06,0.03,0.05,0.989779,0.983022,-74233.396828,0.000000,EUR,PLN
7,8,EUR/USD,Vanilla,1,-1,100000,105000,0.342466,1.050,1.025,0.05,0.03,0.04,0.989779,0.986395,-24200.498097,24360.507873,EUR,USD
8,9,EUR/USD,Vanilla,-1,1,100000,105000,0.482192,1.050,1.025,0.05,0.03,0.04,0.985638,0.980897,-69452.722727,73927.792089,EUR,USD
9,10,EUR/USD,Vanilla,1,-1,100000,105000,0.342466,1.050,1.025,0.05,0.03,0.04,0.989779,0.986395,-24200.498097,24360.507873,EUR,USD
